In [ ]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

#lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
#lista=['SPELLUSDT', 'AXSUSDT', 'ETHUSDT', 'BNBUSDT', 'OGNUSDT', 'CYBERUSDT', '1000PEPEUSDT', 'COMPUSDT', 'ADAUSDT', 'BCHUSDT', 'RUNEUSDT', 'MATICUSDT', 'BLZUSDT', 'HBARUSDT', 'TRBUSDT', 'WLDUSDT', 'MTLUSDT', 'BTCUSDT', 'XRPUSDT', 'OPUSDT', 'PERPUSDT', 'ARBUSDT', 'STMXUSDT', 'CFXUSDT', 'SOLUSDT', 'DOGEUSDT', 'APEUSDT', 'UNFIUSDT', 'LTCUSDT', 'STORJUSDT', 'CRVUSDT']
lista=['STMXUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        #data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data,porcentajeentrada = md.estrategia_trampa(symbol,tp_flag = True)
        data = md.estrategia_haz(symbol, debug=debug)
        #data,porcentajeentrada = md.estrategia_oro('XAU',tp_flag = True)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = imprimo)
        #resultado = md.backtestingsanta(data, plot_flag = imprimo, debug = True)
        ########################################################################
        #if resultado['Return [%]'] < 0: # para santa3 mazmorra
        #if data.disparo.iloc[-1]!=0: # imprime las monedas que estan en posible haz martillo
        #        print(symbol)
        if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


In [1]:
import modulos as md
import numpy as np
import os
import sys
import constantes as cons
lista_filtrada = []
#lista=md.filtradodemonedas ()
lista=['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'ZRXUSDT', 'DOGEUSDT', 'MKRUSDT', 'DOTUSDT', 'TRBUSDT', 'RUNEUSDT', 'SOLUSDT', 'STORJUSDT', 'AVAXUSDT', 'ENJUSDT', 'FILUSDT', 'MATICUSDT', 'COTIUSDT', '1000SHIBUSDT', 'BAKEUSDT', 'DYDXUSDT', 'GALAUSDT', 'APEUSDT', 'OPUSDT', 'LQTYUSDT', 'ARBUSDT', 'LEVERUSDT', '1000PEPEUSDT', 'WLDUSDT', 'BNTUSDT', 'ARKUSDT']
#lista=['ETHUSDT','BCHUSDT','ADAUSDT','ZRXUSDT','TRBUSDT','ENJUSDT','FILUSDT','COTIUSDT','DYDXUSDT','APEUSDT','OPUSDT','LEVERUSDT']## GANADORES
#lista=['ZRXUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        #print(symbol)
        data = md.estrategia_royal(symbol, debug = False, refinado = False, file_source = False, timeframe = '1h')
        resultado=md.backtesting_royal(data, plot_flag=imprimo, symbol=symbol)
        if resultado['Win Rate [%]'] >= 50:
            lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#(data.loc[data['Open Time'] == '2023-10-03 03:00:00'])    


BTCUSDT - Return [%]: -2.8% - # Trades: 6 - Profit Factor: 0.0 - Win Rate [%]: 0.0
ETHUSDT - Return [%]: 2.29% - # Trades: 2 - Profit Factor: 6.521001616730695 - Win Rate [%]: 50.0
BCHUSDT - Return [%]: 1.85% - # Trades: 4 - Profit Factor: 1.5642576427419366 - Win Rate [%]: 50.0
XRPUSDT - Return [%]: -0.04% - # Trades: 8 - Profit Factor: 1.004623161284745 - Win Rate [%]: 25.0
LTCUSDT - Return [%]: 0.4% - # Trades: 5 - Profit Factor: 1.1687790739196342 - Win Rate [%]: 40.0
LINKUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
ADAUSDT - Return [%]: 1.83% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 100.0
BNBUSDT - Return [%]: 1.35% - # Trades: 11 - Profit Factor: 1.5247428530252836 - Win Rate [%]: 36.36363636363637
ATOMUSDT - Return [%]: 0.72% - # Trades: 4 - Profit Factor: 1.8699460702221922 - Win Rate [%]: 25.0
DOGEUSDT - Return [%]: -1.04% - # Trades: 2 - Profit Factor: 0.0 - Win Rate [%]: 0.0
MKRUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor:

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


TRBUSDT - Return [%]: 7.67% - # Trades: 2 - Profit Factor: nan - Win Rate [%]: 100.0
SOLUSDT - Return [%]: -0.15% - # Trades: 4 - Profit Factor: 0.9773282770654963 - Win Rate [%]: 25.0
STORJUSDT - Return [%]: -1.42% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
AVAXUSDT - Return [%]: -0.65% - # Trades: 2 - Profit Factor: 0.0 - Win Rate [%]: 0.0
MATICUSDT - Return [%]: -2.11% - # Trades: 2 - Profit Factor: 0.0 - Win Rate [%]: 0.0
UNFIUSDT - Return [%]: 2.82% - # Trades: 3 - Profit Factor: 2.3408511413067967 - Win Rate [%]: 33.33333333333333
STMXUSDT - Return [%]: 0.02% - # Trades: 7 - Profit Factor: 1.241438267846776 - Win Rate [%]: 28.57142857142857
OPUSDT - Return [%]: -0.99% - # Trades: 5 - Profit Factor: 0.7688083004340397 - Win Rate [%]: 20.0
BNXUSDT - Return [%]: -3.2% - # Trades: 4 - Profit Factor: 0.0 - Win Rate [%]: 0.0
LQTYUSDT - Return [%]: 1.22% - # Trades: 2 - Profit Factor: 1.9380473414791841 - Win Rate [%]: 50.0
ARBUSDT - Return [%]: -0.67% - # Trades: 1 - Profit

In [2]:
lista

['BTCUSDT',
 'ETHUSDT',
 'BCHUSDT',
 'XRPUSDT',
 'LTCUSDT',
 'LINKUSDT',
 'ADAUSDT',
 'BNBUSDT',
 'ATOMUSDT',
 'DOGEUSDT',
 'MKRUSDT',
 'TRBUSDT',
 'SOLUSDT',
 'STORJUSDT',
 'AVAXUSDT',
 'MATICUSDT',
 'UNFIUSDT',
 'STMXUSDT',
 'OPUSDT',
 'BNXUSDT',
 'LQTYUSDT',
 'ARBUSDT',
 'WLDUSDT',
 'BNTUSDT',
 'LOOMUSDT']